In [1]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import requests
import json

import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from io import BytesIO

import time
import json
from ratelimit import limits, sleep_and_retry

### Accessing the wiki Art api

In [2]:
# Load our API access key
access_key = os.getenv('WIKIART_ACCESS_KEY')
secret_code = os.getenv('WIKIART_SECRET_KEY')

In [4]:
login_url = "https://www.wikiart.org/en/Api/2/login"

In [14]:
# Examine the most viewed artists list

base_url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"

try:
    response = requests.get(base_url)
    response.raise_for_status()
    
    result = response.json()
    
    most_visited_df = pd.DataFrame(result.get('data', []))
    
    print("Download complete")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Download complete


In [15]:
login_url = "https://www.wikiart.org/en/Api/2/login"
painting_details_url = "https://www.wikiart.org/en/api/2/Painting"

# Painting details for "The Starry Night"
painting_id = "5772716cedc2cb3880c1907f"  # Alternatively, use the URL slug "the-starry-night-1889"

try:
    # Step 1: Create session
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }
    login_response = requests.get(login_url, params=login_params)
    login_response.raise_for_status()
    
    login_data = login_response.json()
    session_key = login_data.get('SessionKey')
    
    if not session_key:
        raise Exception(f"Failed to obtain session key. Response: {login_data}")

    print(f"Successfully obtained session key: {session_key}")

    # Step 2: Retrieve painting details
    painting_params = {
        "id": painting_id,
        "authSessionKey": session_key  # Pass the session key
    }

    response = requests.get(painting_details_url, params=painting_params)

    if response.status_code == 200:
        painting_data = response.json()
        starry_night_df = pd.DataFrame([painting_data])  # Convert painting data to DataFrame
        print("Download complete")
        display(starry_night_df.head()) 
    elif response.status_code == 404:
        print(f"Painting not found for ID: {painting_id}")
    else:
        print(f"Failed to retrieve painting details for ID: {painting_id}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")
    if hasattr(e, 'response'):
        print(f"Response status code: {e.response.status_code}")
        print(f"Response content: {e.response.text}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully obtained session key: dfaccb0d5dc0
Download complete
                         id             title                    url  \
0  5772716cedc2cb3880c1907f  The Starry Night  the-starry-night-1889   

          artistUrl         artistName                  artistId  \
0  vincent-van-gogh  van Gogh Vincent   57726d82edc2cb3880b486a0   

   completitionYear                                       dictionaries  \
0              1889  [57726b52edc2cb3880ad77e0, 57726b4eedc2cb3880a...   

                       location period  ...          media sizeX sizeY  \
0  Saint-rémy-de-provenceFrance   None  ...  [oil, canvas]  92.1  73.7   

  diameter                                          galleries  \
0     None  [Museum of Modern Art (MoMA), New York City, N...   

                                                tags  \
0  [houses-and-buildings, twilight-and-night, Arl...   

                                         description width  \
0  Van Gogh's night sky is a field of roiling en

In [9]:
starry_night_df['styles']

0    [Post-Impressionism]
Name: styles, dtype: object

In [10]:


def download_image(image_url, save_path):
    try:
        # Send a GET request to the image URL
        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Open the image using PIL
        img = Image.open(BytesIO(response.content))

        # Save the image
        img.save(save_path)
        print(f"Image successfully downloaded and saved to {save_path}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading the image: {e}")
    except IOError as e:
        print(f"Error saving the image: {e}")

# Assuming starry_night_df is the DataFrame containing the painting data
if 'starry_night_df' in locals() and not starry_night_df.empty:
    # Get the image URL from the DataFrame
    image_url = starry_night_df['image'].iloc[0]
    
    # Create a directory to save the image
    save_dir = "downloaded_images"
    os.makedirs(save_dir, exist_ok=True)
    
    # Generate a filename for the image
    filename = f"starry_night_{painting_id}.jpg"
    save_path = os.path.join(save_dir, filename)
    
    # Download and save the image
    download_image(image_url, save_path)
else:
    print("Painting data not available. Please ensure you've successfully retrieved the painting details first.")

# Print the image URL for verification
print(f"Image URL: {image_url if 'image_url' in locals() else 'Not available'}")

Image successfully downloaded and saved to downloaded_images\starry_night_5772716cedc2cb3880c1907f.jpg
Image URL: https://uploads3.wikiart.org/00475/images/vincent-van-gogh/the-starry-night-1889.jpg!Large.jpg


In [11]:
base_url = "https://www.wikiart.org/en/api/2"

def get_session_key():
    login_url = f"{base_url}/login"
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }
    response = requests.get(login_url, params=login_params)
    response.raise_for_status()
    return response.json().get('SessionKey')

def make_api_request(endpoint, params):
    session_key = get_session_key()
    params['authSessionKey'] = session_key
    url = f"{base_url}/{endpoint}"
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def explore_dictionaries_by_group(group):
    print(f"\nExploring Dictionaries by Group {group}")
    try:
        data = make_api_request("DictionariesByGroup", {"group": group})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")

def explore_artists_by_dictionary(group, dict_url):
    print(f"\nExploring Artists by Dictionary - Group: {group}, DictUrl: {dict_url}")
    try:
        data = make_api_request("ArtistsByDictionary", {"group": group, "dictUrl": dict_url})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")
def explore_painting_search(term):
    print(f"\nExploring Painting Search - Term: {term}")
    try:
        data = make_api_request("PaintingSearch", {"term": term})
        print(json.dumps(data, indent=2))
    except Exception as e:
        print(f"Error: {e}")

# Example usage
explore_dictionaries_by_group(1)  # Explore group 1
explore_dictionaries_by_group(2)  # Explore group 2




Exploring Dictionaries by Group 1
{
  "data": [
    {
      "id": "5d230437edc2c9fb74756177",
      "title": "1st Intermediate Period (2181\u20132055 BC)",
      "url": "1st-intermediate-period-2181-2055-bc",
      "group": 1
    },
    {
      "id": "5d23049cedc2c9fb74782eb2",
      "title": "2nd Intermediate Period (1650\u20131550 BC)",
      "url": "2nd-intermediate-period-1650-1550-bc",
      "group": 1
    },
    {
      "id": "5d23052fedc2c9fb747bf1d9",
      "title": "3rd Intermediate Period (1069\u2013664 BC)",
      "url": "3rd-intermediate-period-1069-664-bc",
      "group": 1
    },
    {
      "id": "57726a68edc2ca38801d5111",
      "title": "Abbasid Period (750\u20131258)",
      "url": "abbasid-period-750-1258",
      "group": 1
    },
    {
      "id": "57726a67edc2ca38801d4d81",
      "title": "Abstract Art",
      "url": "abstract-art",
      "group": 1
    },
    {
      "id": "57726a67edc2ca38801d4e69",
      "title": "Abstract Expressionism",
      "url": "abstract

In [12]:
# After exploring dictionaries, we'll use a sample dict_url for Artists by Dictionary
# You'll need to replace 'sample_dict_url' with an actual URL from the DictionariesByGroup response
explore_artists_by_dictionary(1, "baroque")

# Example painting search
explore_painting_search("impressionism")


Exploring Artists by Dictionary - Group: 1, DictUrl: baroque
{
  "data": [
    {
      "id": "62502c309e4363244cf85596",
      "artistName": "Michelangelo Cerquozzi",
      "url": "michelangelo-cerquozzi",
      "lastNameFirst": null,
      "birthDay": "/Date(-58948387200000)/",
      "deathDay": "/Date(-9782640000000)/",
      "birthDayAsString": "102",
      "deathDayAsString": "1660",
      "image": "https://uploads3.wikiart.org/00387/images//h0027-l06020228.jpg!Portrait.jpg",
      "wikipediaUrl": null,
      "dictionaries": [
        "57726a66edc2ca38801d4cd1",
        "57726b4fedc2cb3880ad71d0"
      ],
      "periods": [],
      "series": [],
      "activeYearsStart": null,
      "activeYearsCompletion": null,
      "biography": "",
      "gender": "male",
      "originalArtistName": "",
      "relatedArtists": []
    },
    {
      "id": "625874c99e436338ccb9f54f",
      "artistName": "Simone Pignoni",
      "url": "simone-pignoni",
      "lastNameFirst": null,
      "birthDay

In [ ]:
import time
import requests
import pandas as pd
from ratelimit import limits, sleep_and_retry
from itertools import count

# Rate limits
REQUESTS_PER_SECOND = 4
REQUESTS_PER_HOUR = 400
SESSIONS_PER_HOUR = 10

# API URL and credentials
base_url = "https://www.wikiart.org/en/api/2"

# Decorators to manage rate limits
@sleep_and_retry
@limits(calls=REQUESTS_PER_SECOND, period=1)
def call_api():
    pass

@sleep_and_retry
@limits(calls=REQUESTS_PER_HOUR, period=3600)
def hourly_limit():
    pass

# Fetch session key only once and reuse it
def get_session_key():
    login_url = f"{base_url}/login"
    login_params = {
        "accessCode": access_key,
        "secretCode": secret_code
    }

    try:
        response = requests.get(login_url, params=login_params)
        response.raise_for_status()
        return response.json().get('SessionKey')
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve session key: {e}")
        raise

# Global variable for session key
session_key = None

def make_api_request(endpoint, params, max_retries=3, delay=5):
    global session_key
    if not session_key:
        session_key = get_session_key()  # Fetch session key only once

    for attempt in range(max_retries):
        try:
            call_api()
            hourly_limit()
            
            # Attach the session key to params
            params['authSessionKey'] = session_key
            url = f"{base_url}/{endpoint}"
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            
            # Check if it's a limit issue
            if 'limit exceeded' in str(e).lower():
                print("API rate limit exceeded. Waiting before retrying...")
                time.sleep(60 * 15)  # Wait for 15 minutes
            elif attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Max retries reached. Unable to complete request to {endpoint}.")
                raise

def fetch_all_dictionaries():
    all_dictionaries = []
    
    for group in count(1):  # Start from group 1 and continue indefinitely
        try:
            data = make_api_request("DictionariesByGroup", {"group": group})
            
            if not data:  # If we get an empty response, assume we've reached the end
                break
            
            all_dictionaries.extend(data)
            print(f"Fetched dictionaries for group {group}")
        except Exception as e:
            print(f"Error fetching group {group}: {e}")
            if "Not Found" in str(e):  # If we get a "Not Found" error, assume we've reached the end
                break
            # For other errors, we might want to retry or handle differently
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_dictionaries)
    
    return df

# Usage
dictionaries_df = fetch_all_dictionaries()
print(dictionaries_df)

Fetched dictionaries for group 1
Fetched dictionaries for group 2
Fetched dictionaries for group 3
Fetched dictionaries for group 4
Fetched dictionaries for group 5
Fetched dictionaries for group 6
Fetched dictionaries for group 7
Fetched dictionaries for group 8
Fetched dictionaries for group 9
Fetched dictionaries for group 10
Fetched dictionaries for group 11
Fetched dictionaries for group 12
Fetched dictionaries for group 13
Fetched dictionaries for group 14
Fetched dictionaries for group 15
Fetched dictionaries for group 16
Fetched dictionaries for group 17
Fetched dictionaries for group 18
Fetched dictionaries for group 19
Fetched dictionaries for group 20
Fetched dictionaries for group 21
Fetched dictionaries for group 22
Fetched dictionaries for group 23
Fetched dictionaries for group 24
Fetched dictionaries for group 25
Fetched dictionaries for group 26
Fetched dictionaries for group 27
Fetched dictionaries for group 28
Fetched dictionaries for group 29
Fetched dictionaries fo

In [ ]:
top_styles = [
    "Impressionism",
    "Realism",
    "Romanticism",
    "Expressionism",
    "Post-Impressionism",
    "Baroque",
    "Art Nouveau (Modern)",
    "Surrealism",
    "Symbolism",
    "Abstract Expressionism",
    "Neoclassicism",
    "Naïve Art (Primitivism)",
    "Rococo",
    "Cubism",
    "Northern Renaissance",
    "Academicism",
    "Pop Art",
    "Mannerism (Late Renaissance)",
    "Minimalism",
    "Conceptual Art",
    "Abstract Art",
    "Art Informel",
    "Early Renaissance",
    "Ukiyo-e",
    "Magic Realism",
    "Neo-Expressionism",
    "High Renaissance",
    "Contemporary Realism",
    "Color Field Painting",
    "Orientalism",
    "Lyrical Abstraction",
    "Fauvism",
    "Contemporary",
    "Op Art",
    "Neo-Impressionism",
    "Art-Deco",
]